In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from smooth.adam_general.core.adam import ADAM
import numpy as np
import pandas as pd

# Two-Stage Initialization Tests

Two-stage initialization works by:
1. First running a model with `initial="complete"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods

### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting

In [3]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [4]:
model_optimal = ADAM(model='ANN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,N,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with optimal initial:
Parameters: [2.50250224e-02 9.85091808e+01]
Forecast: [101.13396235 101.13396235 101.13396235]


In [5]:
model_backcasting = ADAM(model='ANN', lags=[12], initial='backcasting', n_iterations=2)
model_backcasting.fit(ts_df)
print('ETS(A,N,N) with backcasting (n_iterations=2):')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with backcasting (n_iterations=2):
Parameters: [0.]
Forecast: [99.61910829 99.61910829 99.61910829]


In [6]:
model_two_stage = ADAM(model='ANN', lags=[12], initial='complete', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,N,N) with complete initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with complete initialization:
Parameters: [0.]
Forecast: [99.61910829 99.61910829 99.61910829]


In [7]:
model_two_stage = ADAM(model='ANN', lags=[12], initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,N,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,N,N) with two-stage initialization:
Parameters: [ 0.         99.61910829]
Forecast: [99.61910829 99.61910829 99.61910829]


### Test 2: Local trend ETS(A,A,N)

In [8]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [9]:
model_optimal = ADAM(model='AAN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with optimal initial:
Parameters: [1.97195505e-01 4.65138729e-03 1.00609474e+02 2.57617847e-01]
Forecast: [165.47911455 166.17224446 166.86537437]


In [10]:
model_optimal = ADAM(model='AAN', lags=[12], initial='complete')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with complete initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with complete initial:
Parameters: [0.20075593 0.        ]
Forecast: [165.26494713 165.89118519 166.51742326]


In [11]:
model_optimal = ADAM(model='AAN', lags=[12], initial='complete')
model_optimal.fit(ts_df)
print('ETS(A,A,N) with complete initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with complete initial:
Parameters: [0.20075593 0.        ]
Forecast: [165.26494713 165.89118519 166.51742326]


In [12]:
model_two_stage = ADAM(model='AAN', lags=[12], initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,A,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,A,N) with two-stage initialization:
Parameters: [ 0.45270534  0.         60.91052144 -0.15393627]
Forecast: [165.09467445 164.94073818 164.78680191]


### Test 3: Seasonal data ETS(A,A,A)

In [13]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [14]:
model_optimal = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,A,A) with optimal initial:')
print('Alpha, Beta, Gamma:', model_optimal.adam_estimated['B'][:3])
print('Forecast:', model_optimal.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: [0.07547122 0.01237746 0.0009072 ]
Forecast: [141.98941932 154.8596331  170.41429601]


In [15]:
model_backcasting = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(A,A,A) with backcasting initial:')
print('Alpha, Beta, Gamma:', model_backcasting.adam_estimated['B'][:3])
print('Forecast:', model_backcasting.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: [0.10465241 0.         0.07079328]
Forecast: [143.82099272 157.40311078 172.98667167]


In [16]:
model_complete = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='complete')
model_complete.fit(ts_df)
print('ETS(A,A,A) with complete initial:')
print('Alpha, Beta, Gamma:', model_complete.adam_estimated['B'][:3])
print('Forecast:', model_complete.   predict(h=12)['mean'].values[:3])

ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: [0.10465241 0.         0.07079328]
Forecast: [143.82099272 157.40311078 172.98667167]


In [17]:
model_two_stage = ADAM(model='AAA', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,A,A) with two-stage initialization:')
print('Alpha, Beta, Gamma:', model_two_stage.adam_estimated['B'][:3])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,A,A) with two-stage initialization:
Alpha, Beta, Gamma: [0.18700916 0.         0.00081227]
Forecast: [150.36584916 164.72565958 177.23997471]


### Test 4: Damped trend ETS(A,Ad,N)

In [18]:
model_optimal = ADAM(model='AAdN', lags=[12], initial='optimal')
model_optimal.fit(ts_df)
print('ETS(A,Ad,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with optimal initial:
Parameters: [ 2.34549076e-04  9.62276905e-06  9.78688799e-01  2.00830255e+02
 -7.42318181e-01]
Forecast: [169.20502405 169.15304469 169.10217306]


In [19]:
model_backcasting = ADAM(model='AAdN', lags=[12], initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(A,Ad,N) with backcasting initial:')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with backcasting initial:
Parameters: [0.00279467 0.         1.        ]
Forecast: [167.37146801 167.20245061 167.0334332 ]


In [20]:
model_two_stage = ADAM(model='AAdN', lags=[12], initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(A,Ad,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(A,Ad,N) with two-stage initialization:
Parameters: [  0.           0.           0.25596358 179.05051374  -0.55409584]
Forecast: [178.85989356 178.85989356 178.85989356]


### Test 5: Multiplicative error ETS(M,N,N)

In [21]:
model_optimal = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(M,N,N) with optimal initial:')
print('Parameters:', model_optimal.adam_estimated['B'])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with optimal initial:
Parameters: [3.29329547e-02 1.89756969e+02]
Forecast: [174.24319564 174.24319564 174.24319564]


In [22]:
model_backcasting = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(M,N,N) with backcasting initial:')
print('Parameters:', model_backcasting.adam_estimated['B'])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with backcasting initial:
Parameters: [0.03312988]
Forecast: [174.19715699 174.19715699 174.19715699]


In [23]:
model_two_stage = ADAM(model='MNN', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(M,N,N) with two-stage initialization:')
print('Parameters:', model_two_stage.adam_estimated['B'])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(M,N,N) with two-stage initialization:
Parameters: [3.30320795e-02 1.06521735e+03]
Forecast: [174.23504989 174.23504989 174.23504989]


### Test 6: Multiplicative seasonal ETS(M,A,M)

In [24]:
model_optimal = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='optimal')
model_optimal.fit(ts_df)
print('ETS(M,A,M) with optimal initial:')
print('Alpha, Beta, Gamma:', model_optimal.adam_estimated['B'][:3])
print('Forecast:', model_optimal.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: [0.11608121 0.00462305 0.00036006]
Forecast: [145.11984075 159.00727681 174.12660471]


In [25]:
model_backcasting = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='backcasting')
model_backcasting.fit(ts_df)
print('ETS(M,A,M) with backcasting initial:')
print('Alpha, Beta, Gamma:', model_backcasting.adam_estimated['B'][:3])
print('Forecast:', model_backcasting.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: [0.09873239 0.         0.06260129]
Forecast: [145.57499447 158.35568597 172.91243136]


In [26]:
model_complete = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='complete')
model_complete.fit(ts_df)
print('ETS(M,A,M) with complete initial:')
print('Alpha, Beta, Gamma:', model_complete.adam_estimated['B'][:3])
print('Forecast:', model_complete.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: [0.09873239 0.         0.06260129]
Forecast: [145.57499447 158.35568597 172.91243136]


In [27]:
model_two_stage = ADAM(model='MAM', lags=[12], distribution='dnorm', initial='two-stage', n_iterations=2)
model_two_stage.fit(ts_df)
print('ETS(M,A,M) with two-stage initialization:')
print('Alpha, Beta, Gamma:', model_two_stage.adam_estimated['B'][:3])
print('Forecast:', model_two_stage.predict(h=12)['mean'].values[:3])

ETS(M,A,M) with two-stage initialization:
Alpha, Beta, Gamma: [0.18577276 0.         0.00096706]
Forecast: [149.58345147 165.00705835 181.99887702]


## Summary

Two-stage initialization successfully:
- Runs a complete backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches